# NLE Assessed Coursework 3: Question 3

For this assessment, you are expected to complete and submit 4 notebook files.  There is 1 notebook file for each question (to speed up load times).  This is notebook 3 out of 4.

Marking guidelines are provided as a separate document.

In order to provide unique datasets for analysis by different students, you must enter your candidate number in the following cell.

In [1]:
candidateno=184514 #this MUST be updated to your candidate number so that you get a unique data sample


In [2]:
#preliminary imports
import sys
sys.path.append(r'\\ad.susx.ac.uk\ITS\TeachingResources\Departments\Informatics\LanguageEngineering\resources')
sys.path.append(r'/Users/juliewe/resources')
sys.path.append(r'/Users/Bayley/Documents/resources')
import re
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from itertools import zip_longest
from sussex_nltk.corpus_readers import ReutersCorpusReader
import random
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import math
import spacy
nlp=spacy.load('en_core_web_sm')
from nltk.corpus import gutenberg

Sussex NLTK root directory is /Users/Bayley/Documents/resources


## Question 3: Named Entity Recognition and Linking (25 marks)

The code below will run the SpaCy system on the text from Persuasion by Jane Austen.  `mysample` contains a 50% sample which is unique to your candidate number.

In [3]:
#Do NOT change the code in this cell.

#preparing corpus

def clean_text(astring):
    #replace newlines with space
    newstring=re.sub("\n"," ",astring)
    #remove title and chapter headings
    newstring=re.sub("\[[^\]]*\]"," ",newstring)
    newstring=re.sub("VOLUME \S+"," ",newstring)
    newstring=re.sub("CHAPTER \S+"," ",newstring)
    newstring=re.sub("\s\s+"," ",newstring)
    #return re.sub("([^\.|^ ])  +",r"\1 .  ",newstring).lstrip().rstrip()
    return newstring.lstrip().rstrip()


def get_sample(sentslist,seed=candidateno):
    random.seed(seed)
    random.shuffle(sentslist)
    testsize=int(len(sentslist)/2)
    return sentslist[testsize:]
    
persuasion=clean_text(gutenberg.raw('austen-persuasion.txt'))
nlp_persuasion=list(nlp(persuasion).sents)

mysample=get_sample(nlp_persuasion)

In [4]:
type(mysample[0])

spacy.tokens.span.Span

a) **Write code** and **extract**:
* the 30 most common strings referring to PEOPLE in `mysample`.
* the 30 most common strings referring to PLACES in `mysample`.

\[6 marks\]

In [7]:
def display_sent(asent):
    headings=["token","lower","lemma","pos","NER"]
    info=[]
    for t in asent:
        info.append([t.text,t.lower_,t.lemma_,t.pos_,t.ent_type_])
    return(pd.DataFrame(info,columns=headings))

def token_func(sents):
    token_list = []
    POS_tags = []
    entity_tags = []
    for sent in sents:
        table = display_sent(sent)
        for item in table['token']:
            token_list += [item]
        for item in table['pos']:
            POS_tags += [item]
        for item in table['NER']:
            entity_tags += [item]
    return token_list, POS_tags, entity_tags

token_list, tag_list, entity_tags = token_func(mysample)

def tag_type(token_list, tag_list, tag_type):
    tag_type_dict = {}
    i = 0
    list1 = []
    while i < len(token_list) - 1:
        if tag_list[i] == tag_type:
            if tag_list[i+1] == tag_type:
                list1 += [(token_list[i] + " " +  token_list[i+1], tag_type)]
                
            else:
                list1 += [(token_list[i], tag_type)]
        i += 1
    return list1


people_dict = {}
people_list1 = tag_type(token_list, entity_tags, 'PERSON')
for item in people_list1:
    if len(item[0]) > 2:
        people_dict[item[0]] = people_dict.get(item[0], 0) + 1
people_list2 = []
for item, count in people_dict.items():
    people_list2 += [(item, count)]
sorted_people_list = sorted(people_list2, key = lambda tup: tup[1], reverse = True)
most_referred_people = sorted_people_list[:30]
print("30 Most common strings referring to PEOPLE", '\n', most_referred_people, '\n')

places_dict = {}
places_list1 = tag_type(token_list, entity_tags, 'LOC') + tag_type(token_list, entity_tags, 'GPE')
for item in places_list1:
    places_dict[item[0]] = places_dict.get(item[0], 0) + 1
places_list2 = []
for item, count in places_dict.items():
    places_list2 += [(item, count)]
sorted_places_list = sorted(places_list2, key = lambda tup: tup[1], reverse = True)
most_referred_places = sorted_places_list[:30]
print("30 Most common strings referring to PLACES", '\n', most_referred_places)

30 Most common strings referring to PEOPLE 
 [('Anne', 206), ('Elliot', 146), ('Wentworth', 95), ('Lady Russell', 78), ('Mary', 67), ('Walter', 67), ('Captain Wentworth', 62), ('Musgrove', 60), ('Charles', 60), ('Russell', 60), ('Elizabeth', 47), ('Harville', 36), ('Benwick', 27), ('Smith', 25), ('Mrs Musgrove', 21), ('Clay', 21), ('Captain Benwick', 20), ('Mrs Clay', 19), ('Charles Hayter', 19), ('Louisa', 18), ('Hayter', 18), ('Shepherd', 18), ("Russell 's", 18), ('Wallis', 15), ('Captain Harville', 14), ('Mrs Smith', 13), ('Anne Elliot', 12), ('Mr Shepherd', 12), ('Lady Dalrymple', 11), ('Dalrymple', 10)] 

30 Most common strings referring to PLACES 
 [('Uppercross', 30), ('Camden Place', 12), ('Place', 12), ('Lyme', 10), ('Kellynch', 9), ('London', 9), ('Indies', 6), ('the West', 5), ('West Indies', 5), ('Plymouth', 3), ('Monkford', 3), ('England', 3), ('Shropshire', 3), ('Cape', 2), ('Westgate Buildings', 2), ('Buildings', 2), ('Ireland', 2), ('Winthrop', 2), ('the East', 1), ('Ea

The function display_sent creates the column names for the table that will be used to store the tokens with corrosponding given tags. For example the token Anne is of type People and would have a corrosponding tag `PER`. The columns lower will be used to translate the tokens such as *Captain* into captain. This is needed as captain can be used as a role, or can be used as a title. The lemma column ensures that any plurals are reverted into the singular form of the word. An example in mysample could be years into year. IOB labels not used here as by using IOB labels, this creates many of the same tags next to eachother which is not wanted, since this could lead it to believe many nouns close could be listed incorrectly as one NE. This function goes through every token in the sentences and adds various data to the information list, returning the table with the tokens, lower case representations, lemmas, pos tags and NER tags

Token_func function creates a list for the token, POS tags and entity tags. For every sentence add the specific values to the specific lists. i.e. add the tokens to the token lists, post tags to the POS tags list and entity tags to entity_tag list.

Tag type function creates a dictionary to store the tag types by going through each token and checks if its corresponding tag type is equivalent to the given tag type. Then checks if there are two tokens in a row with the same tag type and assumes that they are part of the same item i.e. Mr Elliot would both have the same NER tag PERSON, so will be grouped together as one item. If this isnt the case just adds just the singular item to the tag list.

The code then gets a list of all tokens with the tag type 'PERSON' using the function tag_type with the paramter PERSON. Then stores each item, in the list, in a dictionary with a count of how many times they occur. This is needed as we need to find the 30 most common strings, so the number of occurences corrspond to how common the string is. Then adds each person and count into a list of tuples, ordering the list of tuples in descending order of count, having the most common first, printing just the first 30 in the list using the code `sorted_people_list[:30]`

Now we do the similar effect but using the tag LOC (for location). This is because a place is a location. The tag GPE is also used as geo-political entity which means the same as a place.

b) Making reference to specific examples from the text in `mysample`, **discuss** the different types of errors made by the named entity recogniser. \[6 marks\]

One error made the named entity recogniser is that Mrs Musgrove, who is a person (we know this from the title **Mrs**) has been labelled as a location (LOC) or a GPE. We know this as she is outputted above as one of the most common strings referring to a place. This is due to the word `Musgrove` having multiple meaning or in other words having Entity Ambiguity. This is because in this sample Musgrove has been used as a surname for a person, one of its meanings, however the named entity recogniser knows that this is also the name of the `location` **Musgrove park hospital**, therefore has been labelled incorrectly in this instance. Another example of a token being given multiple NER tag's is the token `Benwick`. This was given the NER tag ORG, specifying that is an organisation, yet is also a village and civil parish in the Fenland district of Cambridgeshire, England, so could be given the tag `LOC` or GPE. Another type error made by the named entity recogniser is the token `Smith`. This is because there may be two occurences of the name and may refer to different people, since it is known to be a very common name in England, yet they are both tagged as PERSON so we would not know which one of the people if there was more than one, was being referenced.

In addition, tokens such as `Aunt` and `Anne` which both mean the same thing, i.e. **name variation.** For example these two tokens at a time may be referring to the same person (Anne), however as we do not know the full context of the corpus, so could be possibly mis-tagged. However, even if we knew that `Anne` could be reffered to as `his aunt`, the use of `aunt`, could also be used to refer to other people, for example another female mentioned `Mrs Clay`. This makes it very difficult to link named entities using the named entity recogniser. 

`Name variation` and `entity ambiguity` are the two main problems with `entity linking`. Sorting one of these out, for example sorting out the name variation, increases the problem with ambiguity. this is because by giving a word more like words, when deciding what a word means, there are now even more options that the word could now mean. An example of this could be giving Aunt as another definition for Mrs Clay, the name variation would reduce the problem for name variation, but would increase the problem for entity ambiguity when deciding which meaning to go with when faced with the entity Aunt. The fact that an entity can be of different types and the fact that a string may refer to multiple named entities makes it difficult to link the the classified type and named entity. To reduce ambiguity, techniques such as feature extraction can be used.

In my output, examples of:
- false positives are in Location, Mrs Wallis, Mrs Musgrove and Lyme who are all People
- false negatives are **Cobb** is a location not listed in the locations
- An example of a mis-classification error is `temple` has been listed as a location when talked about `rub her temple` i.e. the case of someone head. Misclassification errors lead to boundary errors when calculating precision and recall.

c) **Design** and **implement** a system to track the locations of characters throughout the story.  For a given PERSON named entity, your system should return a list of time-ordered LOCATIONS for that character.  Test your system using the complete text of "Persuasion" (**not** `mysample`) for at least 3 major characters.   \[13 marks\]

To complete this task an entity linking technqiue must be used. The goal of named entity linking is to either return:
- Conical entities in KB of entry being mentioned
- NIl if entry does not exist

The techniques for entity linking are:

- Step 1: Find candidates within Kb (Knowledge base) for given rnitity mention
- Step 2: Rank candidates to find most probable

When generating candidates, we must address the name variation problem, while finding all potential revelant candidates. This leads to a trade off between precision & recall. We need a high recall so that correct entities is among the candidates, but this hurts precision and efficiency. In my solution only 3 characters will be **tracked**, so efficiency isnt too much of a problem, but I will expect there to be a lack of precision (when compared to recall).

The strategies for generating candidates will include ignoring context, and the mention is an exact match. With more experience strategies such as substring mentions and acroynm mentions can be used, and I will attempt to implement these if possible. By ignoring context, this increases recall, getting as many candidates as possible, while getting the exact match increases precision.

Using string simmularity to generate candidates is a viable method to entity linking. This helps towards the problem with name variations, but at the same time increases the problem with ambiguity. An example of this that I will implement is the `Levenshtein Distance`, also known as the **Minimum Edit Distance** The Levenshtein distance between 2 is the numner of:

- Insertions
- Deletions
- Substitutions.

required to transform one string to the other. A benefit of using this is that, maybe not in this case, but errors in spelling could lead to entity linking. Moreover this helps with entity linking with the use of nicknames, as it can see the two words are similar. An example of Levenshtein is shown below:
![Levenstein Example](leven.gif) <br>
Ranking of candidates is not needed, as we are returning a complete list of locations in order found.

In [11]:
token_list, tag_list, entity_tags = token_func(nlp_persuasion)
people_list1 = tag_type(token_list, entity_tags, 'PERSON')

for item in people_list1:
    if len(item[0]) > 2:
        people_dict[item[0]] = people_dict.get(item[0], 0) + 1
places_dict = {}
places_list1 = tag_type(token_list, entity_tags, 'LOC') + tag_type(token_list, entity_tags, 'GPE')
for item in places_list1:
    if len(item[0]) > 2:
        places_dict[item[0]] = places_dict.get(item[0], 0) + 1

def levenshtein_distance(word1, word2):
    lev_dis = 0
    word1_length = len(word1)
    word2_length = len(word2)
    if word1_length > word2_length:
        lev_dis += word1_length - word2_length
    else:
        lev_dis += word2_length - word1_length
    for i in range(0, min(word1_length, word2_length)):
        if word1[i] != word2[i]:
            lev_dis += 1
    return lev_dis



def location_tracker(person, text):
    location_list = []
    close_words = [person]
    sentences_to_search = []
  
    for key in people_dict.keys():
        if person in key and key not in close_words:
            close_words += [key]
    for sentence in text:
        i=0
        while i < len(sentence) - 2:
            if levenshtein_distance(person, (str(sentence[i]) + " " + str(sentence[i+1])+ " " + str(sentence[i+2]))) < 5 and (str(sentence[i]) + " " + str(sentence[i+1])+ " " + str(sentence[i+2])) not in close_words:
                close_words += [str(sentence[i]) + " " + str(sentence[i+1])+ " " + str(sentence[i+2])]
            elif levenshtein_distance(person, (str(sentence[i]) + " " + str(sentence[i+1]))) < 5 and (str(sentence[i]) + " " + str(sentence[i+1])) not in close_words:
                close_words += [str(sentence[i]) + " " + str(sentence[i+1])]
            elif levenshtein_distance(person, (str(sentence[i]))) < 3 and str(sentence[i]) not in close_words:
                close_words += [str(sentence[i])]
            i = i + 1
    for sentence in text:
        i = 0
        while i < len(sentence) - 2:
            if (str(sentence[i]) + " " + str(sentence[i+1]) + " " + str(sentence[i+2])) in close_words and sentence not in sentences_to_search:
                sentences_to_search += [sentence]
            elif (str(sentence[i]) + " " + str(sentence[i+1])) in close_words and sentence not in sentences_to_search:
                sentences_to_search += [sentence]
            elif (str(sentence[i])) in close_words and sentence not in sentences_to_search:
                sentences_to_search += [sentence]
            i = i+1
    for sentence in sentences_to_search:
        locations_in_sentence = []
        i = 0
        while i < len(sentence):
            if str(sentence[i]) + " " + str(sentence[i+1]) in places_dict.keys():
                locations_in_sentence += [str(sentence[i]) + " " + str(sentence[i+1])]
            elif str(sentence[i]) in places_dict.keys():
                locations_in_sentence += [str(sentence[i])]
            i += 1
        if len(locations_in_sentence) > 0:
            location_list += [locations_in_sentence]
    return location_list

In [12]:
print("Walter Elliot has been at these locations", location_tracker("Walter Elliot", nlp_persuasion), '\n')
print("Russell has been at these locations", location_tracker("Russell", nlp_persuasion), '\n')
print("Charles has been at these locations", location_tracker("Charles", nlp_persuasion ),'\n')

Walter Elliot has been at these locations [['Kellynch'], ['Esq .'], ['Bath'], ['Esq .'], ['Kellynch', 'Somersetshire'], ['Anne', 'Anne'], ['London', 'Bath'], ['Cheshire', 'Dugdale', 'Kellynch', 'Esq .']] 

Russell has been at these locations [['Kellynch'], ['Wentworth', 'Harville', 'Anne', 'Benwick'], ['Kellynch', 'Anne'], ['Harville'], ['Benwick'], ['Anne', 'Captain Wentworth', 'Wentworth'], ['Anne'], ['Anne', 'Bath', 'Uppercross Cottage', 'Cottage', 'Kellynch'], ['Bath'], ['Anne', 'Captain Wentworth', 'Wentworth'], ['Bath'], ['Benwick'], ['Captain Wentworth', "Wentworth 's"], ['Uppercross'], ['Harville', 'Captain Wentworth', 'Wentworth'], ['Anne'], ['Anne'], ['Anne', 'Camden Place', 'Place'], ['Lyme'], ['Anne'], ['Uppercross', 'Captain Wentworth', "Wentworth 's"], ['Anne', 'Anne'], ['Kellynch'], ['Benwick'], ['Anne'], ['Anne'], ['Wallis'], ['Anne'], ['Kellynch', 'Bath'], ['Camden Place', 'Place'], ['Anne'], ['Bath'], ['Uppercross', 'Kellynch'], ['Anne'], ['Anne'], ['Anne'], ['Anne'],

The code works by first of all makeing sure that all the items in the dictionary of people has a length greater than 
2, this is to exclude anything that may be wrongly tagged as a person as peoples names tend be longer than 2. The same is then done for the  dictionary of places/locations.

The next function is used to calcuate the Levenshtein Distance. This finds similar words to the word passed in the location tracker. It does this by creating a list of words which are assumed to be close to the person i.e. *Anne .... England*. We then look for close words for the names of the people for example if `Benwick` is mentioned we also want to look for `Captain Benwick`. The difference in is then length to the value of the Levenshtein Distance and calculates the number of changes required to make one word equal to the other.

The location tracker function starts by finding words that would be close to the person using the Levenshtein Distance
calculated. Here i haved used a distance of 5 to assume tbey are close. By checking for at maximum 3 words in a row this increases the number of possible candidates, increasing the recall. Continues to searchs through the sentences to see where the close words occur, and if any of the close words occur in the sentence, add these sentence to a list of sentences to be searched for locations. Finally goes through each of sentences in sentences-to-search to find locations, checking up to two words to see if the words occur in the dictionary of places. When a location is found, its added to a list for each sentence. If the length of the list of locations is not empty, then add it to the list of total locations for that person.

From my results I can say that my location tracker funciton has a very **high recall**, returing alot of possible candidtes for each person. The reason for a small test on a character such as walter Elliot is due the lack of performace on this machine, along with wanting to see if less common people return lists of a decent size. By analysing the results I can also determine that the precision of my tracker is not very good. This is because I decided to maximise recall, to ensure all possible candidates are considered. I know my precision is bad as Names such as `Mrs Musgrove`, `Anne` and `Captain Wentworth` are retruned within the list of locations. To improve this, you could implement features such as using a large KB to improve precsion, but in turn would reduce recall along with efficiency, which i tried to maximise within this funciton. I do know my function is somewhat precise however when looking at the results of Walter Elliot, with the majoirty of the locations being tagged correctly as well as being recalled.